IMAGE CAPTIONING_DISSERTATION

In [ ]:
#Import libraries
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
%matplotlib inline

import string
import os
import glob
from PIL import Image
from time import time
import keras
from keras import Input, layers
from keras import optimizers



In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
from keras.preprocessing import sequence
from keras.preprocessing import image
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Embedding, Dense, Activation, Flatten, Reshape, Dropout
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam




DATA Loading and Preprocessing

In [ ]:
##download train dataset
!gdown --id "1GSxuTJpgtCoJUOTLLmlJyIri3JXkjYL9"
##download test dataset
!gdown --id "1Uj2jSbSggJfsLVJDDlxvHOUxb0BDuxij"
##download token dataset
!gdown --id "1o6S-v9W2YXnhhlp2CzdLrPvxFyJG8uxb"

In [ ]:
trainpath="/content/Flickr_8k.trainImages.txt"
testpath="/content/Flickr_8k.testImages.txt"
tokenpath="/content/Flickr8k.token.txt"


In [ ]:
doc = open(tokenpath,'r').read()
print(doc[:410])

Creating Dictionary

In [ ]:
descriptions = dict()
for line in doc.split('\n'):
        tokens = line.split()
        if len(line) > 2:
          imageid = tokens[0].split('.')[0]
          imagedesc = ' '.join(tokens[1:])
          if imageid not in descriptions:
              descriptions[imageid] = list()
          descriptions[imageid].append(imagedesc)

In [ ]:
##Text Cleaning
import string
table = str.maketrans('', '', string.punctuation)
for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
        desc = desc_list[i]
        desc = desc.split()
        desc = [word.lower() for word in desc]
        desc = [w.translate(table) for w in desc]
        desc_list[i] =  ' '.join(desc)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
imagepath='/content/drive/MyDrive/ImageDataset/Flicker8k_Dataset/'

In [ ]:

##Visualizing the image and checking its captions
pic = '1000268201_693b08cb0e.jpg'
x=plt.imread(imagepath+pic)
plt.imshow(x)
plt.show()
descriptions['1000268201_693b08cb0e']

In [ ]:
#Creating vocabulary for all Unique words present across
vocabulary = set()
for key in descriptions.keys():
        [vocabulary.update(d.split()) for d in descriptions[key]]
print('Original Vocabulary Size: %d' % len(vocabulary))

In [ ]:
#Saving the image id and its cleaned caption in the same format as it was having a token format
lines = list()
for key, desc_list in descriptions.items():
    for desc in desc_list:
        lines.append(key + ' ' + desc)
new_descriptions = '\n'.join(lines)

In [ ]:
new_descriptions

In [ ]:
#Loading all the train image ids from flickr8k_trainimages in train dataset
doc = open(trainpath,'r').read()
dataset = list()
for line in doc.split('\n'):
    if len(line) > 1:
      identifier = line.split('.')[0]
      dataset.append(identifier)

train = set(dataset)

In [ ]:
#saving the training and test images in the respective variables train_img and test_img
img = glob.glob(imagepath + '*.jpg')
train_images = set(open(trainpath, 'r').read().strip().split('\n'))
train_img = []
for i in img: 
    if i[len(imagepath):] in train_images:
        train_img.append(i)

test_images = set(open(testpath, 'r').read().strip().split('\n'))
test_img = []
for i in img: 
    if i[len(imagepath):] in test_images: 
        test_img.append(i)

In [ ]:
#Loading all the test image ids from flickr8k_trainimages in train dataset
doc = open(trainpath,'r').read()
dataset = list()
for line in doc.split('\n'):
    if len(line) > 1:
      identifier = line.split('.')[0]
      dataset.append(identifier)

test = set(dataset)

In [ ]:
#loading the descriptions of train images in to dictionary and add two tokens called startseq and endseq
desc_train = dict()
for line in new_descriptions.split('\n'):
    tokens = line.split()
    imageid, imagedesc = tokens[0], tokens[1:]
    if imageid in train:
        if imageid not in desc_train:
            desc_train[imageid] = list()
        desc = 'startseq ' + ' '.join(imagedesc) + ' endseq'
        desc_train[imageid].append(desc)

In [ ]:
#creating the list all the training captions
all_train_captions = []
for key, val in desc_train.items():
    for cap in val:
        all_train_captions.append(cap)

In [ ]:
#loading the descriptions of test images in to dictionary and add two tokens called startseq and endseq
test_descriptions = dict()
for line in new_descriptions.split('\n'):
    tokens = line.split()
    imageid, imagedesc = tokens[0], tokens[1:]
    if imageid in test:
        if imageid not in test_descriptions:
            test_descriptions[imageid] = list()
        desc = ' '.join(imagedesc) 
        test_descriptions[imageid].append(desc)
#creating the list all the test captions
all_test_captions = []
for key, val in test_descriptions.items():
    for cap in val:
        all_test_captions.append(cap)

In [ ]:
##checking train and test captions
all_train_captions

In [ ]:
all_test_captions

In [ ]:
#reducing the vocabulary to only those words which occur atleast 8 times in the entire corpus which will make the model more robbust
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1
vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

print('Vocabulary = %d' % (len(vocab)))


In [ ]:
#creating two dictionary to map index to words and viceversa
ix2word = {}
word2ix = {}
ix = 1
for w in vocab:
    word2ix[w] = ix
    ix2word[ix] = w
    ix += 1

vocab_size = len(ix2word) + 1

In [ ]:
##need to find the max length of a caption 
all_desc = list()
for key in desc_train.keys():
    [all_desc.append(d) for d in desc_train[key]]
lines = all_desc
max_length = max(len(d.split()) for d in lines)

print('Description Length: %d' % max_length)

In [ ]:
##200d glove embeddings
##With pretrained glove embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip


In [ ]:
!unzip glove*.zip

In [ ]:
path_to_glove="glove.6B.200d.txt"

In [ ]:
embeddings_index = {} 
with open(path_to_glove, 'r', encoding="utf-8") as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

In [ ]:
##with matrix shape of (1660,200) consisting of the vocabulary created and 200d vector
embedding_dim = 200
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word2ix.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Building model and Training

In [ ]:
##using transfer learning fromincception v3
model = InceptionV3(weights='imagenet')

In [ ]:
new_model = Model(model.input, model.layers[-2].output)

In [ ]:
def preprocess(imagepath):
    img = image.load_img(imagepath, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [ ]:

def encode(image):
    image = preprocess(image) 
    feat_vect = new_model.predict(image) 
    feat_vect = np.reshape(feat_vect, feat_vect.shape[1])
    return feat_vect
 


In [ ]:
##code takes longer time to run
encoding_train = {}
for img in train_img:
  encoding_train[img[len(imagepath):]] = encode(img)
train_features = encoding_train 

    




In [ ]:
encoding_test = {}
for img in test_img:
    encoding_test[img[len(imagepath):]] = encode(img)
test_features=encoding_test

In [ ]:
#defining Model
inputs1 = Input(shape=(2048,))
f1 = Dropout(0.5)(inputs1)
f2 = Dense(256, activation='relu')(f1)

inputs2 = Input(shape=(max_length,))
s1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
s2 = Dropout(0.5)(s1)
s3 = LSTM(256)(s2)

decoder1 = add([f2, s3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [ ]:
model.summary()

In [ ]:
#Model training
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [ ]:
#compiling using categorical cross entropy
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
#Using batches for training the data
def data_generator(descriptions, photos, word2ix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [word2ix[word] for word in desc.split(' ') if word in word2ix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)

            if n==num_photos_per_batch:
                yield ([array(X1), array(X2)], array(y))
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
#Using 30epoch with batch size as 1000
epochs = 30   
batch_size = 10
steps = len(desc_train)//batch_size

generator = data_generator(desc_train, train_features, word2ix, max_length, batch_size)


In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

In [ ]:
earlystop = EarlyStopping(monitor = 'val_loss',min_delta = 0,patience = 3, verbose = 1,restore_best_weights = True)

In [ ]:
model.fit(generator, epochs=epochs, steps_per_epoch=steps,callbacks=[earlystop])

In [ ]:
model.summary

In [ ]:
model.save("/content/drive/MyDrive/Disseration_SharedDrive/model1.h5")

In [ ]:
#Downloading the saved model and using it in future
#Model1download
!gdown --id "1-4fC11e9i3IfSj1IoAnrL8Xv0kUCafUm"

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model=load_model("/content/model1.h5")

In [ ]:
#bleu score evaluation function
def eval_model(model, descriptions, photos, word2ix, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_desc(model, word2ix, photos[key], max_length)
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
	# calculate BLEU score
	
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [ ]:
#preparing the test_set


In [ ]:
# load the model
#model1="/content/model1.h5"
#model = load_model(model1)
# evaluate model
eval_model(model, desc_train, test_features, word2ix, max_length)

In [ ]:
##Greedy and Beam search
def greedySearch(photo):
  in_text = 'startseq'
  for i in range(max_length):
     sequence = [word2ix[w] for w in in_text.split() if w in word2ix]
     sequence = pad_sequences([sequence], maxlen=max_length)
     yhat = model.predict([photo,sequence], verbose=0)
     yhat = np.argmax(yhat)
     word = ix2word[yhat]
     in_text += ' ' + word
     if word == 'endseq':
       break
  final = in_text.split()
  final = final[1:-1]
  final = ' '.join(final)
  return final

In [ ]:
def beam_search_predictions(image, beam_index = 3):
  start = [word2ix["startseq"]]
  start_word = [[start, 0.0]]
  while len(start_word[0][0]) < max_length:
    temp = []
    for s in start_word:
       par_caps = sequence.pad_sequences([s[0]], maxlen=max_length, padding='post')
       preds = model.predict([image,par_caps], verbose=0)
       word_preds = np.argsort(preds[0])[-beam_index:]
       # Getting the top <beam_index>(n) predictions and creating a 
       # new list so as to put them via the model again
       for w in word_preds:
         next_cap, prob = s[0][:], s[1]
         next_cap.append(w)
         prob += preds[0][w]
         temp.append([next_cap, prob])
    start_word = temp
    # Sorting according to the probabilities
    start_word = sorted(start_word, reverse=False, key=lambda l: l[1])
    # Getting the top words
    start_word = start_word[-beam_index:]
  start_word = start_word[-1][0]
  intermediate_caption = [ix2word[i] for i in start_word]
  final_caption = []
  for i in intermediate_caption:
    if i != 'endseq':
      final_caption.append(i)
    else:
      break

  final_caption = ' '.join(final_caption[1:])
  return final_caption

In [ ]:
#Evaluating using greedy and beam search
pic = '2398605966_1d0c9e6a20.jpg'
image = encoding_test[pic].reshape((1,2048))
x=plt.imread(imagepath+pic)
plt.imshow(x)
plt.show()

print("Greedy Search:",greedySearch(image))
print("Beam Search, K = 3:",beam_search_predictions(image, beam_index = 3))
print("Beam Search, K = 5:",beam_search_predictions(image, beam_index = 5))
print("Beam Search, K = 7:",beam_search_predictions(image, beam_index = 7))
print("Beam Search, K = 10:",beam_search_predictions(image, beam_index = 10))

In [ ]:
#Taking few more examples
pic = list(encoding_test.keys())[2]
image = encoding_test[pic].reshape((1,2048))
x=plt.imread(imagepath+pic)
plt.imshow(x)
plt.show()

print("Greedy:",greedySearch(image))
print("Beam Search, K = 3:",beam_search_predictions(image, beam_index = 3))
print("Beam Search, K = 5:",beam_search_predictions(image, beam_index = 5))
print("Beam Search, K = 7:",beam_search_predictions(image, beam_index = 7))

In [ ]:
#Taking few more examples
pic = list(encoding_test.keys())[6]
image = encoding_test[pic].reshape((1,2048))
x=plt.imread(imagepath+pic)
plt.imshow(x)
plt.show()

print("Greedy:",greedySearch(image))
print("Beam Search, K = 3:",beam_search_predictions(image, beam_index = 3))
print("Beam Search, K = 5:",beam_search_predictions(image, beam_index = 5))
print("Beam Search, K = 7:",beam_search_predictions(image, beam_index = 7))
print("Beam Search, K = 10:",beam_search_predictions(image, beam_index = 10))

In [ ]:
#Taking few more examples
pic = list(encoding_test.keys())[11]
image = encoding_test[pic].reshape((1,2048))
x=plt.imread(imagepath+pic)
plt.imshow(x)
plt.show()

print("Greedy:",greedySearch(image))
print("Beam Search, K = 3:",beam_search_predictions(image, beam_index = 3))
print("Beam Search, K = 5:",beam_search_predictions(image, beam_index = 5))
print("Beam Search, K = 7:",beam_search_predictions(image, beam_index = 7))
print("Beam Search, K = 10:",beam_search_predictions(image, beam_index = 10))

In [ ]:
#Taking few more examples
pic = list(encoding_test.keys())[6]
image = encoding_test[pic].reshape((1,2048))
x=plt.imread(imagepath+pic)
plt.imshow(x)
plt.show()

print("Greedy:",greedySearch(image))
print("Beam Search, K = 3:",beam_search_predictions(image, beam_index = 3))
print("Beam Search, K = 5:",beam_search_predictions(image, beam_index = 5))
print("Beam Search, K = 7:",beam_search_predictions(image, beam_index = 7))
print("Beam Search, K = 10:",beam_search_predictions(image, beam_index = 10))

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
for i in range (0,5):
  pic = list(encoding_test.keys())[i]
  image = encoding_test[pic].reshape((1,2048))
  x=plt.imread(imagepath+pic)
  plt.imshow(x)
  plt.show()
  gs=greedySearch(image)
  bs=beam_search_predictions(image, beam_index = 5)
  #print("BLEUscore:",corpus_bleu(gs, bs, weights=(0.3, 0.3, 0.3, 0)))
  print("Greedy:",gs)
  print("Beam Search:",bs)




In [ ]:
#Evaluating using greedy and beam search
pic = '2398605966_1d0c9e6a20.jpg'
image = encoding_test[pic].reshape((1,2048))
x=plt.imread(imagepath+pic)
plt.imshow(x)
plt.show()

print("Greedy Search:",greedySearch(image))
